In [1]:
import pandas
import numpy
from pyArango.connection import *
from pyArango.query import *
import jaydebeapi

<h3>1.設計一個 SQL 資料庫表格，欲將此 CSV 匯入，記錄在 Jupyter Markdown 中。</h3>

In [2]:
data_path = 'data'
csv_file = '%s/movies_data.csv' % data_path
dataset = pandas.read_csv(csv_file)

FileNotFoundError: [Errno 2] File b'data/movies_data.csv' does not exist: b'data/movies_data.csv'

In [ ]:
dataset.head()

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,#Gadhvi (He thought he was Gandhi),2019,109,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
1,#Yaaram,2019,110,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
2,...Aur Pyaar Ho Gaya,1997,147,"Comedy, Drama, Musical",4.7,827,Rahul Rawail,Bobby Deol,Aishwarya Rai Bachchan,Shammi Kapoor
3,...Yahaan,2005,142,"Drama, Romance, War",7.4,1086,Shoojit Sircar,Jimmy Sheirgill,Minissha Lamba,Yashpal Sharma
4,?: A Question Mark,2012,82,"Horror, Mystery, Thriller",5.6,326,Allyson Patel,Yash Dave,Muntazir Ahmad,Kiran Bhatia


In [ ]:
pandas.io.sql.get_schema(dataset.reset_index(), 'movie')

C:\Users\Lunnn\AppData\Local\Temp/ipykernel_7572/3230822329.py:1: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  pandas.io.sql.get_schema(dataset.reset_index(), 'movie')


'CREATE TABLE "movie" (\n"index" INTEGER,\n  "Name" TEXT,\n  "Year" INTEGER,\n  "Duration" INTEGER,\n  "Genre" TEXT,\n  "Rating" REAL,\n  "Votes" INTEGER,\n  "Director" TEXT,\n  "Actor 1" TEXT,\n  "Actor 2" TEXT,\n  "Actor 3" TEXT\n)'

In [ ]:
sql = [
    'CREATE TABLE PUBLIC.MOVIE (',
    '"INDEX" TEXT,',
    'Name TEXT,',
    'Year INTEGER,',
    'Duration INTEGER,',
    'Genre TEXT,',
    'Rating FLOAT,',#real vs float
    'Votes INTEGER,',
    'Director TEXT,',
    'Actor1 TEXT,',
    'Actor2 TEXT,',
    'Actor3 TEXT'
     ');'
]

' '.join(sql)

'CREATE TABLE PUBLIC.MOVIE ( "INDEX" TEXT, Name TEXT, Year INTEGER, Duration INTEGER, Genre TEXT, Rating FLOAT, Votes INTEGER, Director TEXT, Actor1 TEXT, Actor2 TEXT, Actor3 TEXT);'

<h3>2.此 SQL 資料庫表格需要衍生一個索引欄位，規則為：MOV0001，後四碼為序號。</h3>

In [ ]:
empty = []
for i in range(dataset.shape[0]):
    word = 'MOV%04d' %(i+1)
    empty.append(word)
dataset['indexx'] = empty
dataset.head()

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3,indexx
0,#Gadhvi (He thought he was Gandhi),2019,109,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid,MOV0001
1,#Yaaram,2019,110,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor,MOV0002
2,...Aur Pyaar Ho Gaya,1997,147,"Comedy, Drama, Musical",4.7,827,Rahul Rawail,Bobby Deol,Aishwarya Rai Bachchan,Shammi Kapoor,MOV0003
3,...Yahaan,2005,142,"Drama, Romance, War",7.4,1086,Shoojit Sircar,Jimmy Sheirgill,Minissha Lamba,Yashpal Sharma,MOV0004
4,?: A Question Mark,2012,82,"Horror, Mystery, Thriller",5.6,326,Allyson Patel,Yash Dave,Muntazir Ahmad,Kiran Bhatia,MOV0005


<h3>3.撰寫 Python 程式，逐筆產生 SQL INSERT 語句，將此 CSV 匯入該SQL 資料庫表格中。</h3>

In [ ]:
dbConnection = jaydebeapi.connect(
    "org.h2.Driver",
    "jdbc:h2:C:\Projects\DB\data\H2",
    ["SA", ""],
    "C:/Java/h2/bin/h2-1.4.200.jar"
)

dbCursor = dbConnection.cursor()

In [ ]:
_sql = 'TRUNCATE TABLE PUBLIC.MOVIE'
dbCursor.execute(_sql)

In [ ]:
_sql_0 = 'INSERT INTO PUBLIC.MOVIE ("INDEX", NAME, "YEAR", DURATION, GENRE, RATING, VOTES, DIRECTOR, ACTOR1, ACTOR2, ACTOR3) VALUES('

In [ ]:
for i in range(dataset.shape[0]):
    row = dataset.iloc[i].values

    _sql = [
        "'%s'"   % row[-1],#index
        "'%s'"   % row[0].replace('\n', ' ').replace("'", "''"), #name
        '%d'     % row[1],#year
        '%d'     % row[2],#duration
        "'%s'"   % row[3],#genre
        '%f'     % row[4],#rating
        '%d'     % row[5],#votes
        "'%s'"   % row[6].replace('\n', ' ').replace("'", "''"), #DIRECTOR
        "'%s'"   % row[7].replace('\n', ' ').replace("'", "''"), #ACTOR1
        "'%s'"   % row[8].replace('\n', ' ').replace("'", "''"), #ACTOR2
        "'%s'"   % row[9].replace('\n', ' ').replace("'", "''")  #ACTOR3
    ]
    
    _sql_final = _sql_0 + (','.join(_sql)) + ');'
    dbCursor.execute(_sql_final)

<h3>4.設計一個 AQL 資料文件 JSON 格式，欲將此 CSV 匯入，記錄在 Jupyter Markdown 中。</h3>

In [ ]:
data_path = 'data'
csv_file = '%s/movies_data.csv' % data_path
dataset = pandas.read_csv(csv_file)

In [ ]:
aql = ['INSERT {',
       '_key: "%s"'       % dataset.index[i] ,
       ',Name: "%s"'      % dataset["Name"][i],
        ',Year: "%4d"'     % dataset["Year"][i],
        ',Duration: "%d"'  % dataset["Duration"][i],
        ',Genre: "%s"'     % dataset["Genre"][i],
        ',Rating: "%.1f"'  % dataset["Rating"][i],
        ',Votes: "%d"'     % dataset["Votes"][i],
        ',Director: "%s"'  % dataset["Director"][i],
        ',Actor1: "%s"'    % dataset["Actor 1"][i],
        ',Actor2: "%s"'    % dataset["Actor 2"][i],
        ',Actor3: "%s"'    % dataset["Actor 3"][i],
        '} INTO final1 OPTIONS { ignoreErrors: true }']

<h3>5.此 AQL 資料文件之索引規則為：MOV0001，後四碼為序號。</h3>

In [ ]:
empty = []
for i in range(dataset.shape[0]):
    word = 'MOV%04d' %(i+1)
    empty.append(word)
dataset.index = empty
dataset.head()

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
MOV0001,#Gadhvi (He thought he was Gandhi),2019,109,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
MOV0002,#Yaaram,2019,110,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
MOV0003,...Aur Pyaar Ho Gaya,1997,147,"Comedy, Drama, Musical",4.7,827,Rahul Rawail,Bobby Deol,Aishwarya Rai Bachchan,Shammi Kapoor
MOV0004,...Yahaan,2005,142,"Drama, Romance, War",7.4,1086,Shoojit Sircar,Jimmy Sheirgill,Minissha Lamba,Yashpal Sharma
MOV0005,?: A Question Mark,2012,82,"Horror, Mystery, Thriller",5.6,326,Allyson Patel,Yash Dave,Muntazir Ahmad,Kiran Bhatia


<h3>6.撰寫 Python 程式，逐筆產生 AQL INSERT 語句，將此 CSV 匯入該 AQL 資料文件中。</h3>

In [ ]:
db_conn_target = Connection(
        arangoURL='http://127.0.0.1:8529',
        username='emprogria',
        password='alex1314')

In [ ]:
db = db_conn_target['Emprogria']
db_collection = db.collections['final1']

In [ ]:
aql = 'FOR u IN final1  REMOVE u IN final1'

In [ ]:
sub_collections = db.AQLQuery(aql, rawResults=False)

In [ ]:
for i in range(dataset.shape[0]):
    aql = ['INSERT {',
           '_key: "%s"'       % dataset.index[i] ,
           ',Name: "%s"'      % dataset["Name"][i],
           ',Year: "%4d"'     % dataset["Year"][i],
           ',Duration: "%d"'  % dataset["Duration"][i],
           ',Genre: "%s"'     % dataset["Genre"][i],
           ',Rating: "%.1f"'  % dataset["Rating"][i],
           ',Votes: "%d"'     % dataset["Votes"][i],
           ',Director: "%s"'  % dataset["Director"][i],
           ',Actor1: "%s"'    % dataset["Actor 1"][i],
           ',Actor2: "%s"'    % dataset["Actor 2"][i],
           ',Actor3: "%s"'    % dataset["Actor 3"][i],
           '} INTO final1 OPTIONS { ignoreErrors: true }']
    _aql = ''.join(aql)
    sub_collections = db.AQLQuery(_aql, rawResults=False)

<h3>7.撰寫 SQL AGGREATION 語句，計算各 Genre 中之樣本數，記錄在 Jupyter Markdown 中。</h3>

In [ ]:
sql = [
    "SELECT Genre,COUNT(Genre) AS sum",
    "FROM PUBLIC.MOVIE",
    "GROUP BY Genre"
]  
' '.join(sql)

'SELECT Genre,COUNT(Genre) AS sum FROM PUBLIC.MOVIE GROUP BY Genre'

<h3>8.撰寫 Python 程式，調用此 SQL AGGREATION 語句，並顯示結果。</h3>

In [ ]:
dbCursor.execute(' '.join(sql))
resultSet = dbCursor.fetchall()

rows = {
    'GENRE': [],
    'SUM': []
}

for row in resultSet:
    rows['GENRE'].append(row[0])
    rows['SUM'].append(row[1])

df = pandas.DataFrame(data=rows).set_index(['GENRE'])

df.head()

,SUM
GENRE,
clob0: 'Action',170
"clob1: 'Action, Adventure'",17
"clob2: 'Action, Adventure, Biography'",1
"clob3: 'Action, Adventure, Comedy'",37
"clob4: 'Action, Adventure, Crime'",9


<h3>9.撰寫 AQL AGGREATION 語句，計算各 Genre 中之樣本數，記錄在 Jupyter Markdown 中。</h3>

In [ ]:
aql = 'FOR doc IN final1 COLLECT Genre = doc.`Genre` AGGREGATE Total = COUNT(doc.`Genre`) RETURN { "GENRE": Genre, "TOTAL": Total}'

<h3>10.撰寫 Python 程式，調用此 AQL AGGREATION 語句，並顯示結果。</h3>

In [ ]:
sub_collections = db.AQLQuery(aql, rawResults=False, batchSize=dataset.shape[0])
queryResult = sub_collections.response['result']

df = pandas.DataFrame(queryResult).set_index(['GENRE'])

df.head()

,TOTAL
GENRE,
Action,170
"Action, Adventure",17
"Action, Adventure, Biography",1
"Action, Adventure, Comedy",37
"Action, Adventure, Crime",9


<h3>11.撰寫 SQL AGGREATION 語句，計算各 Rating 中之樣本數，記錄在 Jupyter Markdown 中。</h3>

In [ ]:
sql = [
    "SELECT Rating,COUNT(Rating) AS sum",
    "FROM PUBLIC.MOVIE",
    "GROUP BY Rating"
]  
' '.join(sql)

'SELECT Rating,COUNT(Rating) AS sum FROM PUBLIC.MOVIE GROUP BY Rating'

<h3>12.撰寫 Python 程式，調用此 SQL AGGREATION 語句，並顯示結果。</h3>

In [ ]:
dbCursor.execute(' '.join(sql))
resultSet = dbCursor.fetchall()

rows = {
    'RATING': [],
    'SUM': []
}

for row in resultSet:
    rows['RATING'].append(row[0])
    rows['SUM'].append(row[1])

df = pandas.DataFrame(data=rows).set_index(['RATING'])

df.head()

,SUM
RATING,
1.1,1
1.4,1
1.6,2
1.7,5
1.8,4


<h3>13.撰寫 AQL AGGREATION 語句，計算各 Rating 中之樣本數，記錄在 Jupyter Markdown 中。</h3>

In [ ]:
aql = 'FOR doc IN final1 COLLECT Rating = doc.`Rating` AGGREGATE Total = COUNT(doc.`Rating`) RETURN { "RATING": Rating, "TOTAL": Total}'

<h3>14.撰寫 Python 程式，調用此 AQL AGGREATION 語句，並顯示結果。</h3>

In [ ]:
sub_collections = db.AQLQuery(aql, rawResults=False, batchSize=dataset.shape[0])
queryResult = sub_collections.response['result']

df = pandas.DataFrame(queryResult).set_index(['RATING'])

df.head()

,TOTAL
RATING,
1.1,1
1.4,1
1.6,2
1.7,5
1.8,4


<h3>15.撰寫 Python 分析程式，哪些 Director 所拍攝電影之 Rating 最高。</h3>

In [ ]:
ana = dataset.groupby(['Director'])
ana_df = ana['Rating'].aggregate(['mean', 'count'])
ana_df.sort_values(by=['mean'],ascending=False)

,mean,count
Director,,
Saif Ali Sayeed,10.0,1
Sriram Raja,9.7,1
Munni Pankaj,9.4,1
Suhrud Godbole,9.4,1
Arvind Pratap,9.4,1
...,...,...
Navin Batra,1.9,1
Raajeev Walia,1.8,1
Stanley D'Costa,1.8,1


In [ ]:
ana_df.sort_values(by=['count'],ascending=False)

,mean,count
Director,,
David Dhawan,5.226829,41
Mahesh Bhatt,5.753846,39
Shakti Samanta,6.606061,33
Ram Gopal Varma,5.596970,33
Hrishikesh Mukherjee,7.266667,33
...,...,...
Krishnadev Yagnik,7.300000,1
Krishna Shah,6.300000,1
Krishna Mishra,5.600000,1


In [ ]:
dbCursor.close()
dbConnection.close()